<a href="https://colab.research.google.com/github/dtminnick/pytorch-practice/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook completes the requirements for Deep Learning Module 2

Import libraries.

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

Load the MNIST data.

Notes:

transforms.ToTensor() converts the PIL image to a PyTorch tensor and scales pixel values from [0, 255] to [0.0, 1.0].

transforms.Normalize((0.5,), (0.5,)) then normalizes the tensor to have values in the range [-1.0, 1.0], assuming the original data is centered around 0.5.

Normalization is applied during data loading.

In [12]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Get training data.

training_data = datasets.MNIST(root = "./data",
                            train = True,
                            download = True,
                            transform = transform)

# Get testing data.

testing_data = datasets.MNIST(root = "./data",
                           train = False,
                           download = True,
                           transform = transform)

100%|██████████| 9.91M/9.91M [00:01<00:00, 6.08MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 161kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.53MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.25MB/s]


Confirm download of training data.

In [13]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

Confirm download of test data.

In [14]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

Inspect an image from the training set to confirm normalization was applied properly.

Correct normalization will show pixel values roughly in the range -1.0 to 1.0 and a mean that hovers around 0.0, since I normalized at 0.5.

In [16]:
img_tensor, label = training_data[0]

print("Label:", label)
print("Min pixel value:", img_tensor.min().item())
print("Max pixel value:", img_tensor.max().item())
print("Mean pixel value:", img_tensor.mean().item())

Label: 5
Min pixel value: -1.0
Max pixel value: 1.0
Mean pixel value: -0.7246397137641907


Split the data into training, validation and testing sets.

A testing set already exists; randomly take 10,000 images from the training set and reserve them as a validation set.